In [0]:
import torch as th

In [37]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [39]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [40]:
!pip install syft

In [0]:
import syft as sy

In [42]:
hook = sy.TorchHook(th)

W0723 20:00:39.963229 139950777440128 hook.py:98] Torch was already hooked... skipping hooking process


In [43]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

**Lesson : Basic Remote Execution in PySyft**

In [0]:
bob = sy.VirtualWorker(hook, id='bob')

In [45]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [48]:
bob._objects

{40295754305: tensor([1, 2, 3, 4, 5])}

In [49]:
x.location

<VirtualWorker id:bob #objects:1>

In [50]:
x.id_at_location

40295754305

In [51]:
x.id

8157685763

In [52]:
x.owner

<VirtualWorker id:me #objects:0>

In [53]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [54]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [55]:
bob._objects

{}

**Project: Playing with Remote Tensors**

In [0]:
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr = x.send(bob, alice)

In [59]:
x_ptr.child.child

{'alice': (Wrapper)>[PointerTensor | me:94132694421 -> alice:95218117730],
 'bob': (Wrapper)>[PointerTensor | me:96637339652 -> bob:34426468908]}

In [60]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [62]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

**Lesson: Introducing Remote Arithmetic**

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [64]:
x

(Wrapper)>[PointerTensor | me:95364894017 -> bob:62334647772]

In [65]:
y

(Wrapper)>[PointerTensor | me:56917554801 -> bob:56336704128]

In [0]:
z = x + y

In [67]:
z

(Wrapper)>[PointerTensor | me:64504413764 -> bob:32623922857]

In [68]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [69]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:62722774429 -> bob:26280475851]

In [70]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x+y).sum()

In [74]:
z.backward()

(Wrapper)>[PointerTensor | me:12804622098 -> bob:16523172166]

In [0]:
x = x.get()

In [76]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [77]:
x.grad

tensor([1., 1., 1., 1., 1.])

**Project: Learn a Simple Linear Model**

In [0]:
# make a toy dataset
input = th.tensor([[1.,1], [0,1], [1,0], [0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)

In [0]:
weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

In [90]:
for i in range(10):
  
  pred = input.mm(weights)

  loss = ((pred - target)**2).sum()

  loss.backward()

  weights.data.sub_(weights.grad * 0.1)
  weights.grad *= 0

  print(loss.get().data)

tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)
tensor(0.0058)
tensor(0.0037)
tensor(0.0024)
tensor(0.0015)


**Lesson: Garbage Collection and Common Errors**

(Wrapper)>[PointerTensor | me:22925086557 -> bob:86112798480]